#### Prepare Dataset

In [1]:
import deeply.datasets as dd
from   deeply.datasets.util import split as split_dataset

In [2]:
cifar10 = dd.load("cifar10")

In [3]:
train, val = split_dataset(cifar10["train"], splits = (0.7, 0.3))
test = cifar10["test"]

#### Build Model

In [4]:
from deeply.model.dam import DAM

In [5]:
dam = DAM()

ValueError: Input tensors to a BaseModel must come from `tf.keras.Input`. Received: <deeply.ensemble.EnsembleBlock object at 0x15ab0e880> (missing previous layer metadata).

In [ ]:
dam.plot()

In [ ]:
dam.compile()

In [ ]:
# dam.fit(train, validation_data = val, verbose = 2)